In [30]:
import rpTool as rpReader
import rpTool_findpath
import rpToolCache
import rpSBML
import glob
import os
import tarfile
import tempfile
import json

In [2]:
rpreader = rpReader.rpReader()
rpcache = rpToolCache.rpToolCache()
rpreader.deprecatedMNXM_mnxm = rpcache.deprecatedMNXM_mnxm
rpreader.deprecatedMNXR_mnxr = rpcache.deprecatedMNXR_mnxr
rpreader.mnxm_strc = rpcache.mnxm_strc
rpreader.inchikey_mnxm = rpcache.inchikey_mnxm
rpreader.rr_reactions = rpcache.rr_reactions
rpreader.chemXref = rpcache.chemXref
rpreader.compXref = rpcache.compXref
rpreader.nameCompXref = rpcache.nameCompXref

In [3]:
def runFindPathway_hdd(measured_rpsbml_path, input_tar, pathway_id='rp_pathway'):
    dict_global = {}
    with tempfile.TemporaryDirectory() as tmpOutputFolder:
        tar = tarfile.open(fileobj=input_tar, mode='r:xz')
        tar.extractall(path=tmpOutputFolder)
        tar.close()
        measured_rpsbml = rpSBML.rpSBML('measured')
        measured_rpsbml.readSBML(measured_rpsbml_path)
        for sbml_path in glob.glob(tmpOutputFolder+'/*'):
            fileName = sbml_path.split('/')[-1].replace('.sbml', '').replace('.rpsbml', '').replace('.xml', '')
            rpsbml = rpSBML.rpSBML(fileName)
            rpsbml.readSBML(sbml_path)
            found, reactions_score, reactions_std, reactions_ec_score, reactions_ec_std, dict_result = rpTool_findpath.compareRPpathways(measured_rpsbml, rpsbml, True, 'rp_pathway')
            if found:
                dict_global[fileName] = {'reactions_score': reactions_score, 'reactions_std': reactions_std, 'reactions_ec_score': reactions_ec_score, 'reactions_ec_std': reactions_ec_std, 'dict_result': dict_result}
    return dict_global

In [4]:
#try to find all the simulated pathways with the measured ones
all_results = {}
for meas_dir in glob.glob('/home/mdulac/workspace/Galaxy-SynBioCAD/rpOptimise/results/*'):
    sele_res = glob.glob(meas_dir+'/rpSele*')
    if sele_res:
        sele_res = sele_res[0]
        meas_name = meas_dir.split('/')[-1]
        vali_path = '/home/mdulac/workspace/Galaxy-SynBioCAD/rpOptimise/validation/'+meas_name+'.sbml'
        if not os.path.isfile(vali_path):
            print('skipping '+str(vali_path))
            continue
        meas_sbml = glob.glob('/home/mdulac/workspace/Galaxy-SynBioCAD/rpOptimise/validation/*')
        all_results[meas_name] = runFindPathway_hdd(vali_path, open(sele_res, 'rb'), 'rp_pathway')

/home/mdulac/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/mdulac/anaconda3/envs/py36/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [26]:
match_scores = {}
for meas in all_results:
    if all_results[meas]:
        match_scores[meas] = {}
        for reac in all_results[meas]:
            match_scores[meas][reac] = {}
            match_scores[meas][reac]['reac_score'] = all_results[meas][reac]['reactions_score']
            match_scores[meas][reac]['ec_score'] = all_results[meas][reac]['reactions_ec_score']

In [29]:
match_scores

{'measured_64': {'rp_9_2': {'reac_score': 0.05, 'ec_score': 0.75},
  'rp_9_1': {'reac_score': 0.05, 'ec_score': 0.75}},
 'measured_53': {'rp_3_1': {'reac_score': 0.9, 'ec_score': 1.0}},
 'measured_52': {'rp_1_2': {'reac_score': 0.2, 'ec_score': 1.0},
  'rp_1_1': {'reac_score': 0.2, 'ec_score': 1.0}},
 'measured_33': {'rp_2_15': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_24': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_4': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_3': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_8': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_19': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_13': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_21': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_5': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_25': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_12': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_6': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_7': {'reac_score': 0.1, 'ec_score': 0.0},
  'rp_2_14': {'reac_score'

In [32]:
with open('match_scores.json', 'w') as fp:
    json.dump(match_scores, fp)

In [33]:
measured_pathway = rpreader.parseValidation('/home/mdulac/workspace/Galaxy-SynBioCAD/rpOptimise/validation_v0_mnx_removed_16.tsv')

Not equal length between substrate names, their structure or dbref (L-glutamate): 2-amino-1,3-propanediol phosphate <--> InChI=1S/C3H10NO5P/c4-3(1-5)2-9-10(6,7)8/h3,5H,1-2,4H2,(H2,6,7,8) <--> 
Ignoring the folowing product dbref (2-amino-1,3-propanediol phosphate): 
Not equal length between substrate names, their structure or dbref (homoserine): 4-hydroxy-2-ketobutyrate <--> InChI=1S/C4H6O4/c5-2-1-3(6)4(7)8/h5H,1-2H2,(H,7,8)/p-1 <--> 
Ignoring the folowing product dbref (4-hydroxy-2-ketobutyrate): 
Not equal length between substrate names, their structure or dbref (Propionyl-CoA): 3-ketovaleryl-CoA <--> InChI=1S/C26H42N7O18P3S/c1-4-14(34)9-17(36)55-8-7-28-16(35)5-6-29-24(39)21(38)26(2,3)11-48-54(45,46)51-53(43,44)47-10-15-20(50-52(40,41)42)19(37)25(49-15)33-13-32-18-22(27)30-12-31-23(18)33/h12-13,15,19-21,25,37-38H,4-11H2,1-3H3,(H,28,35)(H,29,39)(H,43,44)(H,45,46)(H2,27,30,31)(H2,40,41,42)/p-4/t15-,19-,20-,21+,25-/m1/s1 <--> 
Not equal length between substrate names, their structure or

In [36]:
measured_pathway[2]['steps'].keys()

dict_keys([9, 8, 7, 6, 5, 4, 3, 2, 1])

In [37]:
toret = {}
for meas_id in measured_pathway:
    toret['measured_'+str(meas_id)] = len(measured_pathway[meas_id]['steps'])

In [39]:
print(toret)

{'measured_2': 9, 'measured_4': 9, 'measured_5': 3, 'measured_6': 4, 'measured_7': 4, 'measured_8': 6, 'measured_9': 4, 'measured_10': 8, 'measured_11': 3, 'measured_13': 4, 'measured_14': 6, 'measured_15': 6, 'measured_17': 4, 'measured_18': 6, 'measured_19': 3, 'measured_20': 3, 'measured_21': 9, 'measured_22': 3, 'measured_23': 2, 'measured_24': 4, 'measured_25': 3, 'measured_26': 6, 'measured_27': 4, 'measured_28': 12, 'measured_29': 2, 'measured_30': 2, 'measured_31': 4, 'measured_32': 3, 'measured_33': 2, 'measured_34': 2, 'measured_35': 4, 'measured_36': 4, 'measured_37': 4, 'measured_38': 4, 'measured_39': 4, 'measured_40': 1, 'measured_41': 1, 'measured_42': 2, 'measured_43': 5, 'measured_44': 6, 'measured_45': 3, 'measured_46': 2, 'measured_47': 4, 'measured_48': 1, 'measured_49': 1, 'measured_50': 1, 'measured_51': 1, 'measured_52': 1, 'measured_53': 2, 'measured_54': 6, 'measured_55': 2, 'measured_56': 2, 'measured_57': 5, 'measured_58': 2, 'measured_59': 2, 'measured_60': 

In [20]:
yu

{'measured_64': [('rp_9_2', 0.05, 0.75), ('rp_9_1', 0.05, 0.75)],
 'measured_19': [],
 'measured_53': [('rp_3_1', 0.9, 1.0)],
 'measured_20': [],
 'measured_26': [],
 'measured_47': [],
 'measured_52': [('rp_1_2', 0.2, 1.0), ('rp_1_1', 0.2, 1.0)],
 'measured_33': [('rp_2_15', 0.1, 0.0),
  ('rp_2_24', 0.1, 0.0),
  ('rp_2_4', 0.1, 0.0),
  ('rp_2_3', 0.1, 0.0),
  ('rp_2_8', 0.1, 0.0),
  ('rp_2_19', 0.1, 0.0),
  ('rp_2_13', 0.1, 0.0),
  ('rp_2_21', 0.1, 0.0),
  ('rp_2_5', 0.1, 0.0),
  ('rp_2_25', 0.1, 0.0),
  ('rp_2_12', 0.1, 0.0),
  ('rp_2_6', 0.1, 0.0),
  ('rp_2_7', 0.1, 0.0),
  ('rp_2_14', 0.1, 0.0),
  ('rp_2_2', 0.1, 0.0),
  ('rp_2_10', 0.1, 0.0),
  ('rp_2_9', 0.1, 0.0),
  ('rp_2_22', 0.1, 0.0),
  ('rp_2_18', 0.1, 0.0),
  ('rp_2_23', 0.1, 0.0),
  ('rp_2_17', 0.1, 0.0),
  ('rp_2_16', 0.1, 0.0),
  ('rp_2_20', 0.1, 0.0),
  ('rp_2_1', 0.1, 0.0),
  ('rp_2_11', 0.1, 0.0)],
 'measured_56': [],
 'measured_41': [('rp_1_2', 0.2, 1.0), ('rp_1_1', 0.2, 1.0)],
 'measured_39': [],
 'measured_46': []

In [21]:
all_results['measured_11']

{}

In [6]:
all_results.keys()

dict_keys(['measured_64', 'measured_19', 'measured_53', 'measured_20', 'measured_26', 'measured_47', 'measured_52', 'measured_33', 'measured_56', 'measured_41', 'measured_39', 'measured_46', 'measured_44', 'measured_43', 'measured_30', 'measured_36', 'measured_34', 'measured_10', 'measured_63', 'measured_6', 'measured_42', 'measured_13', 'measured_35', 'measured_61', 'measured_79', 'measured_75', 'measured_8', 'measured_62', 'measured_15', 'measured_71', 'measured_25', 'measured_7', 'measured_59', 'measured_37', 'measured_11', 'measured_38', 'measured_29', 'measured_14', 'measured_72'])

In [7]:
len(all_results)

39

In [12]:
all_results['measured_33'].keys()

dict_keys(['rp_2_15', 'rp_2_24', 'rp_2_4', 'rp_2_3', 'rp_2_8', 'rp_2_19', 'rp_2_13', 'rp_2_21', 'rp_2_5', 'rp_2_25', 'rp_2_12', 'rp_2_6', 'rp_2_7', 'rp_2_14', 'rp_2_2', 'rp_2_10', 'rp_2_9', 'rp_2_22', 'rp_2_18', 'rp_2_23', 'rp_2_17', 'rp_2_16', 'rp_2_20', 'rp_2_1', 'rp_2_11'])

In [11]:
all_results['measured_33']['rp_2_24']

{'reactions_score': 0.1,
 'reactions_std': 0.1,
 'reactions_ec_score': 0.0,
 'reactions_ec_std': 0.0,
 'dict_result': {'species_match': {'MNXM174__64__MNXC3': {'id': 'MNXM174__64__MNXC3',
    'score': 1.0,
    'found': True},
   'MNXM162616__64__MNXC3': {'id': 'CMPD_0000000003__64__MNXC3',
    'score': 0.6,
    'found': True},
   'MNXM2291__64__MNXC3': {'id': 'TARGET_0000000001__64__MNXC3',
    'score': 0.6,
    'found': True}},
  'reactions_match': {'M2': {'reactants': {'MNXM174__64__MNXC3': {'id': 'MNXM174__64__MNXC3',
      'score': 1.0,
      'found': True}},
    'reactants_score': 1.0,
    'products': {'MNXM162616__64__MNXC3': {'id': 'CMPD_0000000003__64__MNXC3',
      'score': 0.6,
      'found': True}},
    'products_score': 0.6,
    'score': 0.2,
    'std': 0.0,
    'found': True,
    'reaction': 'RP2'},
   'M1': {'reactants': {'MNXM162616__64__MNXC3': {'id': 'CMPD_0000000003__64__MNXC3',
      'score': 0.6,
      'found': True}},
    'reactants_score': 0.6,
    'products': {'M

In [3]:
ret = rpreader.parseValidation('/home/mdulac/workspace/Galaxy-SynBioCAD/rpOptimise/validation_v0_mnx_removed_16.tsv')

Not equal length between substrate names, their structure or dbref (L-glutamate): 2-amino-1,3-propanediol phosphate <--> InChI=1S/C3H10NO5P/c4-3(1-5)2-9-10(6,7)8/h3,5H,1-2,4H2,(H2,6,7,8) <--> 
Ignoring the folowing product dbref (2-amino-1,3-propanediol phosphate): 
Not equal length between substrate names, their structure or dbref (homoserine): 4-hydroxy-2-ketobutyrate <--> InChI=1S/C4H6O4/c5-2-1-3(6)4(7)8/h5H,1-2H2,(H,7,8)/p-1 <--> 
Ignoring the folowing product dbref (4-hydroxy-2-ketobutyrate): 
Not equal length between substrate names, their structure or dbref (Propionyl-CoA): 3-ketovaleryl-CoA <--> InChI=1S/C26H42N7O18P3S/c1-4-14(34)9-17(36)55-8-7-28-16(35)5-6-29-24(39)21(38)26(2,3)11-48-54(45,46)51-53(43,44)47-10-15-20(50-52(40,41)42)19(37)25(49-15)33-13-32-18-22(27)30-12-31-23(18)33/h12-13,15,19-21,25,37-38H,4-11H2,1-3H3,(H,28,35)(H,29,39)(H,43,44)(H,45,46)(H2,27,30,31)(H2,40,41,42)/p-4/t15-,19-,20-,21+,25-/m1/s1 <--> 
Not equal length between substrate names, their structure or

In [4]:
ret[33]['steps'][1]['ec_numbers']

['2.1.1.159']

In [5]:
rpreader.validationToSBML('/home/mdulac/workspace/Galaxy-SynBioCAD/rpOptimise/validation_v0_mnx_removed_16.tsv',
                          '/home/mdulac/Downloads/tmp11/')

Not equal length between substrate names, their structure or dbref (L-glutamate): 2-amino-1,3-propanediol phosphate <--> InChI=1S/C3H10NO5P/c4-3(1-5)2-9-10(6,7)8/h3,5H,1-2,4H2,(H2,6,7,8) <--> 
Ignoring the folowing product dbref (2-amino-1,3-propanediol phosphate): 
Not equal length between substrate names, their structure or dbref (homoserine): 4-hydroxy-2-ketobutyrate <--> InChI=1S/C4H6O4/c5-2-1-3(6)4(7)8/h5H,1-2H2,(H,7,8)/p-1 <--> 
Ignoring the folowing product dbref (4-hydroxy-2-ketobutyrate): 
Not equal length between substrate names, their structure or dbref (Propionyl-CoA): 3-ketovaleryl-CoA <--> InChI=1S/C26H42N7O18P3S/c1-4-14(34)9-17(36)55-8-7-28-16(35)5-6-29-24(39)21(38)26(2,3)11-48-54(45,46)51-53(43,44)47-10-15-20(50-52(40,41)42)19(37)25(49-15)33-13-32-18-22(27)30-12-31-23(18)33/h12-13,15,19-21,25,37-38H,4-11H2,1-3H3,(H,28,35)(H,29,39)(H,43,44)(H,45,46)(H2,27,30,31)(H2,40,41,42)/p-4/t15-,19-,20-,21+,25-/m1/s1 <--> 
Not equal length between substrate names, their structure or

Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miri

Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find envipath in self.miriam_header for species
Cannot find envipath in sel

Cannot find envipath in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find envipath in self.miriam_header for species
Cannot find envipath in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find envipath in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Some species are not referenced by a MNX id and will be ignored
Need all the species to have a MNX ID
Need all the species to have a MNX ID
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header fo

Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find envipath in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find envipath in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find envipath in self.miriam_header for species
Cannot find cco in self.miriam_header for compartment
Cannot find cco in se

...

Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miri

Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find envipath in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self

Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in

Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find lipidmaps in self.miriam_header for species
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in self.miriam_header for compartment
Cannot find cco in s

{}

In [5]:
all_results.keys()

dict_keys(['measured_64', 'measured_19', 'measured_53', 'measured_20', 'measured_26', 'measured_47', 'measured_52', 'measured_33', 'measured_56', 'measured_41', 'measured_39', 'measured_46', 'measured_44', 'measured_43', 'measured_30', 'measured_36', 'measured_34', 'measured_10', 'measured_63', 'measured_6', 'measured_42', 'measured_13', 'measured_35', 'measured_61', 'measured_79', 'measured_75', 'measured_8', 'measured_62', 'measured_15', 'measured_71', 'measured_25', 'measured_7', 'measured_59', 'measured_37', 'measured_11', 'measured_38', 'measured_29', 'measured_14', 'measured_72'])

In [13]:
for meas in all_results:
    for step in all_results[meas]:
        try:
            print(all_results[meas][step]['dict_result']['ec_match'])
        except KeyError:
            pass

{'M4': {'score': 1.0, 'found': True, 'id': 'RP4'}, 'M3': {'score': 0.0, 'found': False, 'id': None}, 'M2': {'score': 1.0, 'found': True, 'id': 'RP2'}, 'M1': {'score': 1.0, 'found': True, 'id': 'RP1'}}
{'M4': {'score': 1.0, 'found': True, 'id': 'RP4'}, 'M3': {'score': 0.0, 'found': False, 'id': None}, 'M2': {'score': 1.0, 'found': True, 'id': 'RP2'}, 'M1': {'score': 1.0, 'found': True, 'id': 'RP1'}}
{'M2': {'score': 1.0, 'found': True, 'id': 'RP2'}, 'M1': {'score': 1.0, 'found': True, 'id': 'RP2'}}
{'M1': {'score': 1.0, 'found': True, 'id': 'RP1'}}
{'M1': {'score': 1.0, 'found': True, 'id': 'RP1'}}
{'M2': {'score': 0.0, 'found': False, 'id': None}, 'M1': {'score': 0.0, 'found': False, 'id': None}}
{'M2': {'score': 0.0, 'found': False, 'id': None}, 'M1': {'score': 0.0, 'found': False, 'id': None}}
{'M2': {'score': 0.0, 'found': False, 'id': None}, 'M1': {'score': 0.0, 'found': False, 'id': None}}
{'M2': {'score': 0.0, 'found': False, 'id': None}, 'M1': {'score': 0.0, 'found': False, 'id'

In [16]:
all_results['measured_33']['rp_2_15']['dict_result'].keys()

dict_keys(['species_match', 'reactions_match', 'ec_match'])